In [1]:
import pandas as pd
import tensorflow as tf

def normalize(dataset):
    seriesIndices = ['hours']
    for i in range(1,31):
        seriesIndices.append(f"hours_l{i}")
    dataset[seriesIndices] = dataset[seriesIndices].apply(lambda x: (x - x.mean()) / (x.std()))
    return dataset


ROWS = 10 ** 8
train = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/training_set.csv",nrows=ROWS)
dropList = []
for i in range(1,31):
    dropList.append(f"employees_l{i}")
#train = train.drop(dropList,axis=1)
train = train.dropna()
#train = normalize(train)

#val = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/validation_set.csv",nrows=ROWS/2,skiprows=ROWS)
#test = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/testing_set.csv",nrows=10)
val = train[:int(ROWS*0.2)]
test = train[int(ROWS*0.2):int(ROWS*0.3)]
train = train[int(ROWS*0.3):]




In [2]:
train.columns

Index(['prov_id', 'employee_id', 'job_title', 'pay_type', 'date', 'hours',
       'week', 'day_of_week', 'hours_l1', 'employees_l1', 'hours_l2',
       'employees_l2', 'hours_l3', 'employees_l3', 'hours_l4', 'employees_l4',
       'hours_l5', 'employees_l5', 'hours_l6', 'employees_l6', 'hours_l7',
       'employees_l7', 'hours_l8', 'employees_l8', 'hours_l9', 'employees_l9',
       'hours_l10', 'employees_l10', 'hours_l11', 'employees_l11', 'hours_l12',
       'employees_l12', 'hours_l13', 'employees_l13', 'hours_l14',
       'employees_l14', 'hours_l15', 'employees_l15', 'hours_l16',
       'employees_l16', 'hours_l17', 'employees_l17', 'hours_l18',
       'employees_l18', 'hours_l19', 'employees_l19', 'hours_l20',
       'employees_l20', 'hours_l21', 'employees_l21', 'hours_l22',
       'employees_l22', 'hours_l23', 'employees_l23', 'hours_l24',
       'employees_l24', 'hours_l25', 'employees_l25', 'hours_l26',
       'employees_l26', 'hours_l27', 'employees_l27', 'hours_l28',
      

In [3]:
# %%
# ========================================
#          PREP DATAFRAMES
# ========================================
#train = normalize(train)
#val = normalize(val)
train_inputs, train_labels = train.drop(['prov_id','employee_id','date','hours','week'], axis=1), train.filter(['hours'])
val_inputs, val_labels = val.drop(['prov_id','employee_id','date','hours','week'], axis=1), val.filter(['hours'])
test_inputs, test_labels = test.drop(['prov_id','employee_id','date','hours','week'], axis=1), test.filter(['hours'])

print(train_inputs)
print(train_labels)

          job_title  pay_type  day_of_week  hours_l1  employees_l1  hours_l2  \
32885957         35         2            6      0.00           5.0      0.00   
32885958         35         2            5      0.00           5.0      6.10   
32885959         35         2            4      6.10           5.0      6.00   
32885960         35         2            3      6.00           5.0      6.10   
32885961         35         2            2      6.10           5.0      0.00   
...             ...       ...          ...       ...           ...       ...   
99999995          3         3            3      7.65          32.0      7.55   
99999996          3         3            2      7.55          32.0      0.50   
99999997          3         3            1      0.50          34.0      0.50   
99999998          3         3            1      7.86          27.0      7.81   
99999999          3         3            0      7.81          27.0      6.55   

          employees_l2  hours_l3  emplo

In [4]:
train_inputs.dtypes
print(train_inputs['job_title'].unique())
print(train_inputs.shape)
print(val_inputs.shape)
#print(val_inputs['job_title'].unique())
#print(test_inputs['job_title'].unique())

[35  3 11 33 34 12 14 16]
(60893388, 70)
(20000000, 70)


In [5]:
def expand_one_hot(labels,dataset):
    outList = []
    for label in labels:  
        col = dataset[label]
        ###Generate a dict for all unique values (Don't waste space encoding non important job id's)
        map = {}
        index = 0
        for element in col.unique():
            map[element] = index
            index += 1
        col = col.map(map)
        tensor = tf.one_hot(col,len(col.unique()))
        outList.append(tensor)
        dataset = dataset.drop(columns=[label])
    
    outList.append(dataset)
    output = tf.concat(outList,1)
    return output
    

In [6]:
train_inputs = expand_one_hot(['job_title','pay_type','day_of_week'],train_inputs)
val_inputs = expand_one_hot(['job_title','pay_type','day_of_week'],val_inputs)
#train_inputs = train_inputs.drop(['job_title','pay_type','day_of_week'],axis=1)
#val_inputs = val_inputs.drop(['job_title','pay_type','day_of_week'],axis=1)
test_inputs = expand_one_hot(['job_title','pay_type','day_of_week'],test_inputs)

In [7]:
train_inputs

<tf.Tensor: shape=(60893388, 85), dtype=float32, numpy=
array([[1.  , 0.  , 0.  , ..., 0.05, 0.27, 0.08],
       [1.  , 0.  , 0.  , ..., 0.05, 0.27, 0.08],
       [1.  , 0.  , 0.  , ..., 0.05, 0.27, 0.08],
       ...,
       [0.  , 1.  , 0.  , ..., 0.12, 0.12, 0.15],
       [0.  , 1.  , 0.  , ..., 0.13, 0.14, 0.15],
       [0.  , 1.  , 0.  , ..., 0.13, 0.14, 0.15]], dtype=float32)>

In [8]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [9]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [10]:
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 256
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [11]:
trainSet = tf.data.Dataset.from_tensor_slices((train_inputs,train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valSet = tf.data.Dataset.from_tensor_slices((val_inputs,val_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
testSet = tf.data.Dataset.from_tensor_slices((test_inputs,test_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(trainSet)

<BatchDataset shapes: ((None, 85), (None, 1)), types: (tf.float32, tf.float64)>


In [12]:
import numpy as np
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(16),
        tf.keras.layers.Dense(1),
    ])
    
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [13]:
import os
checkpoint_dir = './training_checkpointsBigModel'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True)
]

In [ ]:
model.fit(trainSet, epochs=20, callbacks=callbacks, validation_data=valSet)

Epoch 1/20
237865/237865 [==============================] - 21441s 90ms/step - loss: 10.5421 - mean_absolute_error: 2.2830 - val_loss: 11.7491 - val_mean_absolute_error: 2.4570
Epoch 2/20
237865/237865 [==============================] - 16716s 70ms/step - loss: 10.2613 - mean_absolute_error: 2.2288 - val_loss: 11.4821 - val_mean_absolute_error: 2.3944
Epoch 3/20
237865/237865 [==============================] - 16877s 71ms/step - loss: 10.1719 - mean_absolute_error: 2.2115 - val_loss: 11.4097 - val_mean_absolute_error: 2.4055
Epoch 4/20
237865/237865 [==============================] - 26855s 113ms/step - loss: 10.1059 - mean_absolute_error: 2.1994 - val_loss: 11.1169 - val_mean_absolute_error: 2.3884
Epoch 5/20
 43173/237865 [====>.........................] - ETA: 5:46:27 - loss: 10.5424 - mean_absolute_error: 2.3196

In [14]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
eval_loss, eval_acc = model.evaluate(testSet)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

39063/39063 [==============================] - 366s 9ms/step - mean_absolute_error: 2.5386 - loss: 11.7877
Eval loss: 11.787664413452148, Eval Accuracy: 2.538590669631958
